## Converting goodreads_book_authors from json to csv

In [1]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime, timedelta, date
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sqlite3
import dask.bag as db
import json
import dask.dataframe as dd

In [3]:
json_bag = db.read_text('goodreads_book_authors.json').map(json.loads)
print(json_bag.take(3))

({'average_rating': '3.98', 'author_id': '604031', 'text_reviews_count': '7', 'name': 'Ronald J. Fields', 'ratings_count': '49'}, {'average_rating': '4.08', 'author_id': '626222', 'text_reviews_count': '28716', 'name': 'Anita Diamant', 'ratings_count': '546796'}, {'average_rating': '3.92', 'author_id': '10333', 'text_reviews_count': '5075', 'name': 'Barbara Hambly', 'ratings_count': '122118'})


In [4]:
authors = json_bag.to_dataframe()

In [5]:
authors = authors.compute()

In [6]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829529 entries, 0 to 829528
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   average_rating      829529 non-null  object
 1   author_id           829529 non-null  object
 2   text_reviews_count  829529 non-null  object
 3   name                829529 non-null  object
 4   ratings_count       829529 non-null  object
dtypes: object(5)
memory usage: 31.6+ MB


In [7]:
authors.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [8]:
authors.to_csv('goodreads_book_authors.csv')

## Unnesting the 'Authors" column in rombook data

In [9]:
json_bag = db.read_text('goodreads_books_romance.json').map(json.loads)
print(json_bag.take(3))

({'isbn': '', 'text_reviews_count': '4', 'series': [], 'country_code': 'US', 'language_code': '', 'popular_shelves': [{'count': '4', 'name': 'to-read'}, {'count': '1', 'name': 'ibooks'}, {'count': '1', 'name': 'favorite-authors'}, {'count': '1', 'name': 'may-2017-dr-reads'}, {'count': '1', 'name': 'sports-romance'}, {'count': '1', 'name': 'series-i-like'}, {'count': '1', 'name': 'romance'}, {'count': '1', 'name': 'favorites'}, {'count': '1', 'name': 'f-f'}, {'count': '1', 'name': 'hockey-romance'}, {'count': '1', 'name': 'ebook'}], 'asin': '', 'is_ebook': 'true', 'average_rating': '3.86', 'kindle_asin': '', 'similar_books': [], 'description': "Secrets. Sometimes keeping them in confidence is a good thing. Other times secrets can slowly allow a woman's soul to rot.\nWhitney Beaupre has been hiding a big secret for years, one that's beginning to wear her down both on and off the ice. Pretending to be something she's not is exhausting. Wanting to be free but afraid to break out of her pri

In [10]:
rombook = json_bag.to_dataframe()

In [11]:
rombook = rombook.compute()

In [12]:
rombook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335449 entries, 0 to 335448
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   isbn                  335449 non-null  object
 1   text_reviews_count    335449 non-null  object
 2   series                335449 non-null  object
 3   country_code          335449 non-null  object
 4   language_code         335449 non-null  object
 5   popular_shelves       335449 non-null  object
 6   asin                  335449 non-null  object
 7   is_ebook              335449 non-null  object
 8   average_rating        335449 non-null  object
 9   kindle_asin           335449 non-null  object
 10  similar_books         335449 non-null  object
 11  description           335449 non-null  object
 12  format                335449 non-null  object
 13  link                  335449 non-null  object
 14  authors               335449 non-null  object
 15  publisher        

In [13]:
#rom_authors = pd.read_csv('goodreads_rombook_data.csv')
#rom_authors.head()

In [14]:
#rom_authors.info()

In [15]:
nested_cols = [col for col in rombook.columns if rombook[col].apply(lambda x: isinstance(x, (dict, list))).any()]
print(nested_cols)

['series', 'popular_shelves', 'similar_books', 'authors']


In [16]:
#Step 1: Extract the nested columns
rom_authors_exploded = rombook.explode(['authors'], ignore_index=True)
# Step 2: Convert dicts to columns
rom_authors_expanded = pd.concat([rom_authors_exploded.drop(columns=['authors']),
                         rom_authors_exploded['authors'].apply(pd.Series)], axis=1)

In [17]:
rom_authors_expanded.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,author_id,role
0,,4,[],US,,"[{'count': '4', 'name': 'to-read'}, {'count': ...",,true,3.86,,...,2017,https://www.goodreads.com/book/show/34883016-p...,https://images.gr-assets.com/books/1493525974m...,34883016,5,56135087,Playmaker: A Venom Series Novella,Playmaker: A Venom Series Novella,5807700,
1,,21,[811663],US,en-US,"[{'count': '598', 'name': 'to-read'}, {'count'...",B01BLJGA9S,true,4.23,B01BLJGA9S,...,,https://www.goodreads.com/book/show/29074693-p...,https://s.gr-assets.com/assets/nophoto/book/11...,29074693,149,46079519,"Prowled Darkness (Dante's Circle, #7)","Prowled Darkness (Dante's Circle, #7)",5360266,
2,1597371289,8,[],US,eng,"[{'count': '16215', 'name': 'classics'}, {'cou...",,false,3.99,B0083Z3O8Y,...,2005,https://www.goodreads.com/book/show/3209316-emma,https://s.gr-assets.com/assets/nophoto/book/11...,3209316,42,3360164,Emma,Emma,1265,
3,,27,[938303],US,en-GB,"[{'count': '25', 'name': 'to-read'}, {'count':...",B01HX6PENG,true,4.31,B01HX6PENG,...,,https://www.goodreads.com/book/show/30838933-g...,https://s.gr-assets.com/assets/nophoto/book/11...,30838933,139,51437308,"Guardian Cougar (Finding Fatherhood, #2)","Guardian Cougar (Finding Fatherhood, #2)",90411,
4,,27,[938303],US,en-GB,"[{'count': '25', 'name': 'to-read'}, {'count':...",B01HX6PENG,true,4.31,B01HX6PENG,...,,https://www.goodreads.com/book/show/30838933-g...,https://s.gr-assets.com/assets/nophoto/book/11...,30838933,139,51437308,"Guardian Cougar (Finding Fatherhood, #2)","Guardian Cougar (Finding Fatherhood, #2)",14356708,


In [18]:
rom_authors_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401040 entries, 0 to 401039
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   isbn                  401040 non-null  object
 1   text_reviews_count    401040 non-null  object
 2   series                401040 non-null  object
 3   country_code          401040 non-null  object
 4   language_code         401040 non-null  object
 5   popular_shelves       401040 non-null  object
 6   asin                  401040 non-null  object
 7   is_ebook              401040 non-null  object
 8   average_rating        401040 non-null  object
 9   kindle_asin           401040 non-null  object
 10  similar_books         401040 non-null  object
 11  description           401040 non-null  object
 12  format                401040 non-null  object
 13  link                  401040 non-null  object
 14  publisher             401040 non-null  object
 15  num_pages        

In [19]:
rom_authors_expanded.to_csv('rom_books.csv')